In [1]:
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io
import os
from PyPDF2 import PdfFileWriter, PdfFileReader

In [17]:
#splits full report into pages
inputpdf = PdfFileReader(open("report.pdf", "rb"))
for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open(f"pages/report-page{i:03}.pdf", "wb") as outputStream:
        output.write(outputStream)

In [18]:
#reports list of images
file_list = []
for path, subdirs, files in os.walk("pages"):  # change depending on system
    for file in files:
        a = os.path.join(file)
        file_list.append(a)
file_list = sorted(file_list)

First five files ['report-page000.pdf', 'report-page001.pdf', 'report-page002.pdf', 'report-page003.pdf', 'report-page004.pdf']


In [19]:
tool = pyocr.get_available_tools()[0]
lang = tool.get_available_languages()[0]

In [20]:
image = []
text = []

In [32]:
for file in file_list:
    pdf = Image(filename="pages/"+file, resolution=300)
    jpg = pdf.convert('jpeg')
    img_page = Image(image=jpg)
    image.append(img_page.make_blob('jpeg'))

In [33]:
for img in image: 
    txt = tool.image_to_string(
        PI.open(io.BytesIO(img)),
        lang=lang,
        builder=pyocr.builders.TextBuilder())
    text.append(txt)

In [34]:
with open("report.txt", "w") as file:
    for page in text:
        file.write(page)